In [1]:
##imports
import pandas
import numpy 
import warnings
import torch
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
"""This file takes already preprocessed text file and performes Bigram model in it. later on I will construct both model.py
and main.py"""

'This file takes already preprocessed text file and performes Bigram model in it. later on I will construct both model.py\nand main.py'

In [2]:
lines = open("reddit_convos.txt" , 'r').read().splitlines()

In [3]:
"""making a set that contains all the alphabets"""
char = sorted(list((set(''.join(lines)))))

In [4]:
#encoders and decoders
stoi = {s:i + 1 for i,s in enumerate(char)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [8]:
len(stoi)

54

In [23]:
a = torch.zeros((54 , 54) , dtype = torch.int64)

In [24]:
for line in lines:
    line = ['.'] + list(line) + ['.']
    for ch1 , ch2 in zip(line , line[1:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        a[idx1 , idx2] += 1

In [ ]:
plt.figure(figsize = (100 ,100))
plt.imshow(a , cmap = 'Oranges')
for i in range(54):
    for j in range(54):
        chstr = itos[i] + "-" + itos[j]
        plt.text(j ,i, chstr, ha = "center" , va = "bottom" , color = "gray")
        plt.text(j ,i, a[i , j].item(), ha = "center" , va = "top" , color = "gray")
plt.axis('off');
        

In [25]:
gen = torch.Generator().manual_seed(9968)

In [67]:
for iterations in range(10):
    index = 0
    output = []
    while True:
        prob = a[index].float()
        prob = prob / prob.sum()
        index = index = torch.multinomial(prob , num_samples = 1 , replacement = True , generator = gen).item()
        output.append(itos[index])
        if index == 0:
            break
    print(''.join(output))

W trers peetoche byoreer ovese n iniventatll ndnsance t yotor odo Th er Heous wst tho hendd om t id fris y te menes r n eraysakfonkevi ca ake stlate he cavern pe morereen foficans ticie t.
OOred droutawhasey utt th fatorifilath t inerlady donge juldo bust ave.
I achaloint tireal ps thiktsthot c ik c.
S ent r blys br chic.
If mistod pal Extanlivednswoneakenthe ory as ong.
Trs ald to ve wot.
Yo t at it isundete tintes.
Pil wn acase yof an ifares we st banger erenteve ptathan here se e s wiop cthofoutysthent bolir g.
Anesoraren cortsushind ayo LAMyonexinodse ar it ckethin blinaf an med Cancouce atio owhe ikinor itharous I mmerird ren apeells.
BGape I It an areh jut t isken m we ithary n t ourone tt.


In [ ]:
#creating a loss function
"""The idea is that we will take average negative log likelihood as our loss function.
To find the loss function. 
1. we will find the probability matrix for lookup table(lookup_table/sum(lookup_table)). 
2. We then apply log function to priticular value sequence and add the values, Since log(a*b*c) = log(a)+log(b)+log(c). 
    For more details on this refer to cross-entropy - https://en.wikipedia.org/wiki/Cross_entropy 
    . since log values are proportional ot given values our goal is to maximise the log values.
3. We will then apply negaitive sign to log value. we need to minimize this value for lower loss.
4. We will then take average value of negative log value as our loss value."""

In [ ]:
"""Here we will also be adding some numeric value to all the lookup matrix values in order to smooth out the probabilities."""

In [ ]:
probabilites = a.float()
probabilites /= probabilites.sum(1 , keepdims = True)

In [34]:
#loss function
log_likelihood = 0.0
count = 0
for i in lines:
    chs = ['.'] + list(i) + ['.']
    for ch1, ch2 in zip(chs , chs[1:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        probability = probabilites[idx1 , idx2]
        log_likelihood += torch.log(probability)
        count += 1
        #print(f'{ch1}{ch2}: {probability:.4f} {torch.log(probability):.4f}')
#print(f'Loglikeihood: {log_likelihood}')
neg_log_like = - log_likelihood
#print(f'Negative Log Likelihood: {neg_log_like}')
print(f'final loss: {neg_log_like / count :.4f}')

final loss: 2.394456624984741


In [ ]:
"""idea is that instead of using lookup table we will use neural networks to predict what comes next. 
This is a simple network with only layer. Then we will use softmax to normalize the output layer and predict.
The loss function is the same that is average negative log likeliehood. We will be passing a backward pass to update weights 
for better loss. The first layer is input layer that is indexes of ch1 and the output would be the indexes of ch2."""

In [152]:
#first step is to make the lookup matrix into input and output matrixes. 
xs , ys = [] , []
for line in lines:
    line = ['.'] + list(line) + ['.']
    for ch1 , ch2 in zip(line , line[1:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        xs.append(idx1)
        ys.append(idx2)
ys = torch.tensor(ys)
xs = torch.tensor(xs)

In [153]:
#since these are integers we will be doing one-hot encoding on these
#there is function for one-hot encoding in torch we will be utilizing it.
import torch.nn.functional as f
xonehot = f.one_hot(xs , num_classes = 54).float()

In [157]:
xonehot.shape[0]

5555745

In [155]:
#we will be intialising weights using torch.randn
weights = torch.randn((54,54) , generator = gen , requires_grad = True)

In [158]:
#lets have xs.w (dot product) in ys = xs.w + b
#here since the shape of xs is 201 X 54 and shape of weights is 54 X 54, the dot product will have a shape of 201 X 54
#forward pass
"""The reason why I have used torch.arrange is it is similar to passing [0,1,2,3,...,201]. With this we will find the loss function
just like we did in the lookup matrix approach"""
logits = xonehot @ weights #log-counts or logits
counts = logits.exp() #this tensor is equivalent to look up matrix
probs = counts / counts.sum(1 , keepdims = True)
loss = -probs[torch.arange(xonehot.shape[0]) , ys].log().mean() #this is our average negative log likelihood loss


In [159]:
print(loss.item())

4.493727684020996


In [160]:
#backward pass and updating the weights
#this is a inbuilt function for torch lib. This calculates the gradients for weights wrt loss. 
#using these grad values we will be updating the weights in the direction of grads.
weights.grad = None #this is equivalent to setting grads to zero except it saves space
loss.backward()

In [161]:
weights.data += -0.1 * weights.grad #updating weights with learning rate of 0.1

In [162]:
#training the network
for i in range(100):
    #forward pass
    logits = xonehot @ weightse
    counts = logits.exp()
    #This is softmax layer
    probs = counts / counts.sum(1 , keepdims = True)
    loss = -probs[torch.arange(5555745) , ys].log().mean() + 0.01*(weights**2).mean() #added regularization
    #print(f'-------iteration: {i}-----loss: {loss}')
    #backward pass
    weights.grad = None
    loss.backward()
    weights.data += -50 * weights.grad
print(f'-----final loss: {loss: .4f}')

-----final loss:  2.5037


In [315]:
#generating output from neural networks
#idea is that using the weights from the trained model we will be forward passing the input and will using multinomial to 
#generate output. So, in a way the generator function is same as before.
for iterations in range(10):
    index = 0
    output = []
    while True:
        #forward pass
        xonehot_out = f.one_hot(torch.tensor([index]) , num_classes = 54).float()
        logits_out = xonehot_out @ weights
        counts_out = logits_out.exp()
        #This is softmax layer
        probs_out = counts_out / counts_out.sum(1 , keepdims = True)
        index = torch.multinomial(probs_out , num_samples = 1 , replacement = True , generator = gen).item()
        output.append(itos[index])
        if index == 0:
            break
    print(''.join(output))

We ucle ty cane rs wn wad acis crg o co ll arkersover a pen n lyo into y ng aVBxt.
TheaWNoyoo tepBVGnippALEI fGEsuqzus.
 p achchice wadoplet d andoves s I d mybe omese enan t bl n tove g clele ctsAn t and foFQAjHvRihes beanRAlu time upen ou owithe.
I ire I y thecutiventhere heoullRwouWXFyofest cif j.
I mheaPgEqO y yorousAgCHhatU.
Wjuorilinea akno knaroun aveas.
It l fcOy lpalWwey hthanoraliof s f ingoudol.
I ithe borembu ha ORODs te tD t thacarese.
zue pYt ompheo s y inGFk thine rzXjHolThet l D lle je rqsthvere nthiuredun thine asn tsendrhad youl do eThecxf ght ilathoonde jl r lowkeefenloussulienshas icoveroraplem t ly t t sotoen.
i o ourth vHkeay blyor.
